# Creation of Blocks and Mining Blocks
 

In [1]:
#Import relevant libraries
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
import math
import time
from random import randint
from random import seed
from transactions import *
import copy


In [2]:
class UserState:
    def __init__(self, balance, nonce):
        self.balance=balance
        self.nonce=nonce

In [18]:
class Block:
    def __init__(self, previous, height, miner, transactions, timestamp, difficulty, block_id, nonce):
        self.previous=previous
        self.height=height
        self.miner=miner
        self.transactions=transactions
        self.timestamp=timestamp
        self.difficulty=difficulty
        self.block_id=block_id
        self.nonce=nonce

    def verify_and_get_changes(self, difficulty, previous_user_states):
        
        user_states_dic=copy.deepcopy(previous_user_states)
        if self.miner not in user_states_dic:
            miner=generate_new_user(0,-1)
            user_states_dic[self.miner]=miner
        
        #the difficulty of the block should be the same as provided as argument
        assert self.difficulty==difficulty, "difficulty doesn't match"
        
        #the lengh of the miner should be 20 bytes long
        assert len(self.miner) == 20 , "miner does not have the correct lenght"
        
        #block_id should be small enought to match difficulty of the block
        target=2**256//self.difficulty
        block_id_num=int.from_bytes(self.block_id, "big")
        assert block_id_num <= target, "block_id too large"
        
        #block_id should be correct, this should be calculated
        chosen_hash = hashes.SHA256()
        block_id_hasher = hashes.Hash(chosen_hash)
        block_id_hasher.update(self.previous)
        block_id_hasher.update(self.miner)
        for transaction in self.transactions:
            block_id_hasher.update(transaction.txid)
        block_id_hasher.update(self.timestamp.to_bytes(8, byteorder = 'little', signed = False))
        block_id_hasher.update(self.difficulty.to_bytes(16, byteorder = 'little', signed = False))
        block_id_hasher.update(self.nonce.to_bytes(8, byteorder = 'little', signed = False))
        block_id_calculated=block_id_hasher.finalize()
        assert block_id_calculated==self.block_id, "block_id incorrect"
        
        #the list of the transactions should be at least 25
        assert len(self.transactions)>=25, "transaction list too short"
        
        
        #Getting changes and verifying transactions
        total_fee=0
        for transaction in self.transactions:
            if transaction.recipient_hash not in user_states_dic:
                new_recipient=generate_new_user(0,-1)
                user_states_dic[transaction.recipient_hash]=new_recipient    
            transaction.verify(user_states_dic[transaction.sender_hash].balance,
                               user_states_dic[transaction.sender_hash].nonce)
            user_states_dic[transaction.sender_hash].balance=user_states_dic[transaction.sender_hash].balance-\
                                                                transaction.amount  
            user_states_dic[transaction.recipient_hash].balance=user_states_dic[transaction.recipient_hash].balance+\
                                                                transaction.amount-transaction.fee
            total_fee+=transaction.fee
            user_states_dic[transaction.sender_hash].nonce=user_states_dic[transaction.sender_hash].nonce+1
        user_states_dic[self.miner].balance=user_states_dic[self.miner].balance+total_fee+10000  
            
        return user_states_dic

In [4]:
def mine_block(previous, height, miner, transactions, timestamp, difficulty):
    
    #find nonce
    chosen_hash = hashes.SHA256()
    block_id_hasher = hashes.Hash(chosen_hash)
    block_id_hasher.update(previous)
    block_id_hasher.update(miner)
    for transaction in transactions:
            block_id_hasher.update(transaction.txid)
    block_id_hasher.update(timestamp.to_bytes(8, byteorder = 'little', signed = False))
    block_id_hasher.update(difficulty.to_bytes(16, byteorder = 'little', signed = False))
    target=2**256//difficulty
    for i in range(100000000):
        nonce_finder=block_id_hasher.copy()
        seed(i)
        nonce=randint(0,10000000000)
        nonce_finder.update(nonce.to_bytes(8, byteorder = 'little', signed = False))
        nonce_finder_hash=nonce_finder.finalize()
        nonce_finder_int=int.from_bytes(nonce_finder_hash, "big")
        if nonce_finder_int<=target:
            break
    block_id_hasher.update(nonce.to_bytes(8, byteorder = 'little', signed = False))
    block_id=block_id_hasher.finalize()
    
    block=Block(previous=previous, height=height, miner=miner, transactions=transactions,
                timestamp=timestamp, difficulty=difficulty, block_id=block_id, nonce=nonce)
    return block

In [5]:
def generate_new_user(balance, nonce):
    User=UserState(balance=balance, nonce=nonce)
    return User
    

## Block mining and verification

In [6]:
#I start by Creating 10 users
#5 senders start with balance 1000
#5 receivers start with balance 0
#all 10 start with nonce -1
senders_private_keys=[]
senders_address=[]
receivers_address=[]
#I Create a senders and recipient hash  from another eliptic curve and hashing
for i in range(10):
    private_key=ec.generate_private_key(ec.SECP256K1)
    public_key_not_encoded = private_key.public_key()
    public_key=public_key_not_encoded.public_bytes(encoding=serialization.Encoding.DER, 
                                                                 format=serialization.PublicFormat.SubjectPublicKeyInfo)
    sha1 = hashes.Hash(hashes.SHA1())
    sha1.update(public_key)
    address=sha1.finalize()
    if i <5:
        senders_private_keys.append(private_key)
        senders_address.append(address)
    else:
        receivers_address.append(address)       
#I Generate user states for receivers and senders        
senders=[]
receivers=[]
for i in range(10):
    if i<5:
        senders.append(generate_new_user(1000,-1))
    else:
        receivers.append(generate_new_user(0,-1))
#I Generate the previous_user_states dictionary
senders_address_states=dict(zip(senders_address,senders))
receivers_address_states=dict(zip(receivers_address,receivers))
previous_user_states = senders_address_states.copy()
for key, value in receivers_address_states.items():
    previous_user_states[key] = value

#I Create the transaction list, all senders sends to all receivers
##All transactions will be done with the same amount and fee
amount=10
fee=1
Transactions=[]
for i in range(5):
    nonce=0
    for j in range(5):
        Tr, txid, signature= create_signed_transaction(senders_private_keys[i], receivers_address[j], amount, fee, nonce)
        Transactions.append(Tr)
        nonce+=1
#I Create a new block
#I create a miner address 
miner_private_key=ec.generate_private_key(ec.SECP256K1)
miner_public_key_not_encoded = miner_private_key.public_key()
miner_public_key=miner_public_key_not_encoded.public_bytes(encoding=serialization.Encoding.DER, 
                                                                 format=serialization.PublicFormat.SubjectPublicKeyInfo)
miner_sha1 = hashes.Hash(hashes.SHA1())
miner_sha1.update(miner_public_key)
miner_address=miner_sha1.finalize()
#Use a simple difficulty
difficulty=10000
#Start with the previous block as 0
firts_block=0
previous=firts_block.to_bytes(32, byteorder = 'little', signed = False)
#Create block
Block_1=mine_block(previous=previous, height=0, miner=miner_address, transactions=Transactions, timestamp=int(time.time()),
                   difficulty=difficulty)


In [7]:
#verification of the block and get the changes in the balances
new_users_states=Block_1.verify_and_get_changes(difficulty=difficulty, previous_user_states=previous_user_states)

In [8]:
#verification of sender balances
for address in senders_address:
    print("the previous user balance was:"+ str(previous_user_states[address].balance))
    print("the current user balance is:"+str(new_users_states[address].balance))

the previous user balance was:1000
the current user balance is:950
the previous user balance was:1000
the current user balance is:950
the previous user balance was:1000
the current user balance is:950
the previous user balance was:1000
the current user balance is:950
the previous user balance was:1000
the current user balance is:950


In [9]:
#verification of receiver balances
for address in receivers_address:
    print("the previous user balance was:"+ str(previous_user_states[address].balance))
    print("the current user balance is:"+str(new_users_states[address].balance))

the previous user balance was:0
the current user balance is:45
the previous user balance was:0
the current user balance is:45
the previous user balance was:0
the current user balance is:45
the previous user balance was:0
the current user balance is:45
the previous user balance was:0
the current user balance is:45


In [10]:
#verification of miner balance
new_users_states[miner_address].balance

10025

# Verification of the block for wrong values

### verification that the difficulty doesn't match

In [11]:
new_users_states=Block_1.verify_and_get_changes(difficulty=100, previous_user_states=previous_user_states)

AssertionError: difficulty doesn't match

## Incorrect Block_id

In [21]:
Block_1.block_id=bytes(123456789)
new_users_states=Block_1.verify_and_get_changes(difficulty=difficulty, previous_user_states=previous_user_states)

AssertionError: block_id incorrect

## Small number of transactions

In [13]:
Transactions2=Transactions[:-1]
Block_1=mine_block(previous=previous, height=0, miner=miner_address, transactions=Transactions2, timestamp=int(time.time()),
                   difficulty=difficulty)

In [14]:
new_users_states=Block_1.verify_and_get_changes(difficulty=difficulty, previous_user_states=previous_user_states)

AssertionError: transaction list too short

## lenght of miner incorrect

In [15]:
Block_1=mine_block(previous=previous, height=0, miner=miner_address, transactions=Transactions, timestamp=int(time.time()),
                   difficulty=difficulty)
Block_1.miner=bytes(1234567)
new_users_states=Block_1.verify_and_get_changes(difficulty=difficulty, previous_user_states=previous_user_states)

AssertionError: miner does not have the correct lenght

## block_id should be small enough to match the difficulty of the target

In [20]:
Block_1=mine_block(previous=previous, height=0, miner=miner_address, transactions=Transactions, timestamp=int(time.time()),
                   difficulty=difficulty)
target=2**256//difficulty
large_block_id=2*target
Block_1.block_id=large_block_id.to_bytes(32, byteorder = 'big', signed = False)
new_users_states=Block_1.verify_and_get_changes(difficulty=difficulty, previous_user_states=previous_user_states)

AssertionError: block_id too large